In [ ]:
import tensorflow as tf
import keras
import numpy as np
import pandas as pd

In [ ]:
data=pd.read_csv('/content/drive/My Drive/dataset/datasets_483_982_spam.csv',encoding='latin-1')

In [ ]:
data

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


In [ ]:
data.drop('Unnamed: 2',axis=1,inplace=True)

In [ ]:
data.drop('Unnamed: 3',axis=1,inplace=True)

In [ ]:
data.drop('Unnamed: 4',axis=1,inplace=True)

In [ ]:
labels=[]

In [ ]:
for i in data['v1']:
  labels.append(i)

In [ ]:
labels

In [ ]:
data1=[]

In [ ]:
for i in data['v2']:
  data1.append(i)


In [ ]:
data1

In [ ]:
data2=[]

In [ ]:
for i in data1:
  data2.append(i.lower())

In [ ]:
data2

In [ ]:
import re

In [ ]:
def preprocess_sentence(w):
  w = re.sub(r"([?.!,¿])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)
  w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
  w = w.strip()
 # w = '<start> ' + w + ' <end>'
  return w

In [ ]:
data3=[]

In [ ]:
for i in data2:
  data3.append(preprocess_sentence(i))

In [ ]:
data3

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [ ]:
max_len=256
max_words=10000


In [ ]:
tokenizer=Tokenizer(num_words=10000)
tokenizer.fit_on_texts(data3)
sequences=tokenizer.texts_to_sequences(data3)
word_index=tokenizer.word_index

In [ ]:
sequences

In [ ]:
texts=pad_sequences(sequences,maxlen=max_len,padding='post')

In [ ]:
texts

In [ ]:
new_df=data['v1'].replace(['ham','spam'],[1,0])

In [ ]:
final=[]

In [ ]:
for i in labels:
  if i=='ham':
    final.append(1)
  else:
    final.append(0)


In [ ]:
len(final)

In [ ]:
len(texts)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_data,val_data,train_label,val_label=train_test_split(texts,final,test_size=0.2)

In [ ]:
train_dataset=tf.data.Dataset.from_tensor_slices((train_data,train_label))
val_dataset=tf.data.Dataset.from_tensor_slices((val_data,val_label))

In [ ]:
dataset=tf.data.Dataset.from_tensor_slices((texts,final))

In [ ]:
train_dataset=train_dataset.batch(64).shuffle(5000)
val_dataset=val_dataset.batch(64).shuffle(1000)

In [ ]:
next(iter(train_dataset))

(<tf.Tensor: shape=(64, 256), dtype=int32, numpy=
 array([[ 186,   22,    2, ...,    0,    0,    0],
        [ 254, 1502,   41, ...,    0,    0,    0],
        [   1,  339,   12, ...,    0,    0,    0],
        ...,
        [ 132,  125, 1073, ...,    0,    0,    0],
        [6641,    6, 6642, ...,    0,    0,    0],
        [ 891,   64,  604, ...,    0,    0,    0]], dtype=int32)>,
 <tf.Tensor: shape=(64,), dtype=int32, numpy=
 array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
        0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1,
        0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       dtype=int32)>)

In [ ]:
model=tf.keras.models.Sequential([
    tf.keras.layers.Embedding(10000,256,input_length=256),
    tf.keras.layers.Conv1D(filters=30,kernel_size=4,strides=2,padding='valid'),
    tf.keras.layers.GRU(30,return_sequences=True),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(2)
])

In [ ]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 256, 256)          2560000   
_________________________________________________________________
conv1d_16 (Conv1D)           (None, 127, 30)           30750     
_________________________________________________________________
gru_17 (GRU)                 (None, 127, 30)           5580      
_________________________________________________________________
flatten_5 (Flatten)          (None, 3810)              0         
_________________________________________________________________
dense_9 (Dense)              (None, 2)                 7622      
Total params: 2,603,952
Trainable params: 2,603,952
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])

In [ ]:
model.fit(train_dataset,epochs=30,validation_data=val_dataset)